In [1]:
!pip install pyomo
!pip install gurobipy
!pip install xpress

In [1]:
# problem data
coins = {
    'penny': 1,
    'nickel': 5,
    'dime': 10,
    'quarter': 25
}

In [4]:
import pyomo
from pyomo.environ import *

def make_change(amount, coins):
    model = ConcreteModel()
    model.x = Var(coins.keys(), domain=NonNegativeIntegers)
    
    model.amount = Constraint(expr = sum(model.x[c]*coins[c] for c in coins.keys()) == amount)
    
    model.total = Objective(expr = sum(model.x[c] for c in coins.keys()), sense=minimize)

    SolverFactory('gurobi').solve(model)
    return {c: int(model.x[c]()) for c in coins.keys()} 
            

In [5]:
import os
os.environ['GRB_LICENSE_FILE']="C:\\Users\\D054070\\Anaconda3\\Lib\\site-packages\\gurobipy\\gurobi.lic"
            
make_change(1034, coins)

{'penny': 4, 'nickel': 1, 'dime': 0, 'quarter': 41}

In [6]:
import gurobipy as gp
def make_change_gurobi(amount, coins):
    model = gp.Model('coin-changer-gurobi')
    x = model.addVars(coins.keys(), vtype=gp.GRB.INTEGER,name='x')
    model.addConstr(sum(x[c]*coins[c] for c in coins.keys()) == amount)
    model.setObjective(sum(x[c] for c in coins.keys()), sense=gp.GRB.MINIMIZE)
   
    model.optimize()
    return {c: int(x[c].x) for c in coins.keys()} 
            

In [7]:
make_change_gurobi(1034, coins)

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x2ce44f57
Variable types: 0 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 46 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.600000000000e+01, best bound 4.600000000000e+01, gap 0.0000%


{'penny': 4, 'nickel': 1, 'dime': 0, 'quarter': 41}

In [8]:
import xpress as xp
def make_change_xpress(amount, coins):
    model = xp.problem('coin-changer-xpress')
    x = {(c): xp.var(vartype=xp.integer,name='x(' + c + ')') for c in coins.keys()}
    model.addVariable(x)
    model.addConstraint(sum(x[c]*coins[c] for c in coins.keys()) == amount)
    model.setObjective(sum(x[c] for c in coins.keys()), sense=xp.minimize)
    model.solve()
    return {c: int(model.getSolution(x[c])) for c in coins.keys()} 

In [9]:
make_change_xpress(1034, coins)

Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('c:/Users/D054070/Anaconda3/lib/site-packages/xpress/license/community-xpauth.xpr')
FICO Xpress v8.14.2, Community, solve started 17:01:25, Oct 11, 2022
Heap usage: 381KB (peak 381KB, 252KB system)
Minimizing MILP noname using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
         1 rows            4 cols            4 elements         4 globals
Presolved problem has:
         1 rows            3 cols            3 elements         3 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 410KB (peak 417KB, 252KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] 

{'penny': 4, 'nickel': 1, 'dime': 0, 'quarter': 41}